<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/IMT_Lab_DecisionTree_KBest_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Lab: Decision Tree, LabelEncode, SelectKBest

Aprenda neste Lab:

1. Empregar o modelo de Árvore de Decisão
2. Para modelos de Árvore de Decisão, é necessário normalizar os dados?
3. Podemos criar modelos mais eficientes com menos features?

# Caso: **Propensão de Compra de Clientes por Telemarketing**

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Os dados refere-se a campanhas de marketing direto de uma instituição bancária portuguesa. As campanhas de marketing foram baseadas em telefonemas e visavam buscar a conversão dos clientes para compra de um produto (depósito bancário), sendo `y` ('yes') ou não ('no') o resultado da aquisição ou não pelo cliente. Os clientes podem então ser avaliados para direcionar um reforço de campanha, por exemplo, fazendo uma nova ligação para clientes com propensão 'yes', mas que ainda não adquiriram o produto. Modelos semelhantes são empregados também em campanhas de *churn* de clientes.

Os dados para construção do modelo estão [aqui](https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/bank-full.csv) e aqui os [novos dados](https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/bank-new.csv) a serem avaliados.




# Roteiro

Neste laboratório:

1. Crie um modelo de Árvore de Decisão que leva em consideração todos os atributos não normalizados
2. Crie um modelo de Árvore de Decisão que leva em consideração todos os atributos normalizados
3. Compare os dois modelos
4. Crie um modelo de Árvore de Decisão que leva em consideração apenas os 8 melhores atributos (não normalizados) para predição
5. Compare os resultados com o do primeiro modelo
6. Faça uma predição para novos casos




# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Aquisição dos Dados

In [ ]:
df = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/bank-full.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# Faça o `LabelEncoder` dos dados

Mesmo para modelos de Árvore de Decisão, o `scikit-learn` requer rótulos numéricos. Aqui vamos aplicar o LabelEncoder também ao atributo objetivo `y` (o que seria opcional).

Dicas:

* Consulte as notas de aula ou a documentação do `LabelEncoder`, diferentemente do `HotOneEncoder` ele opera por atributo
* Crie um dicionário para salvar as transformações aplicadas para cada atributo (por que?)
* Se quiser, persista esse dicionário

In [ ]:
from sklearn.preprocessing import LabelEncoder

LabelEncoder_dict = {}

for column in df.select_dtypes(exclude='number'):
  label_encode = LabelEncoder()
  df[column] = label_encode.fit_transform(df[column])
  LabelEncoder_dict[column] = label_encode

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


In [ ]:
LabelEncoder_dict

{'job': LabelEncoder(),
 'marital': LabelEncoder(),
 'education': LabelEncoder(),
 'default': LabelEncoder(),
 'housing': LabelEncoder(),
 'loan': LabelEncoder(),
 'contact': LabelEncoder(),
 'month': LabelEncoder(),
 'poutcome': LabelEncoder(),
 'y': LabelEncoder()}

In [ ]:
LabelEncoder_dict['job'].transform(['entrepreneur'])

array([2])

In [ ]:
LabelEncoder_dict['job'].inverse_transform([4])

array(['management'], dtype=object)

In [ ]:
import joblib

# Salvando
joblib.dump(LabelEncoder_dict, 'LabelEncoder_dict.pkl')

['LabelEncoder_dict.pkl']

In [ ]:
del LabelEncoder_dict

In [ ]:
LabelEncoder_dict = joblib.load('LabelEncoder_dict.pkl')

Q1. Qual o código gerado para job management, job student e education unknown?


In [ ]:
pd.DataFrame({'job':LabelEncoder_dict['job'].inverse_transform(df.job.unique()),'job_label':df.job.unique()})

,job,job_label
0,management,4
1,technician,9
2,entrepreneur,2
3,blue-collar,1
4,unknown,11
5,retired,5
6,admin.,0
7,services,7
8,self-employed,6
9,unemployed,10


In [ ]:
pd.DataFrame({'education':LabelEncoder_dict['education'].inverse_transform(df.education.unique()),'education_label':df.education.unique()})

,education,education_label
0,tertiary,2
1,secondary,1
2,unknown,3
3,primary,0


# Construa um modelo de Árvore de Decisão (dados não normalizados)

Empregue a separação de dados de treinamento e teste como abaixo e os parâmetros padrão de um modelo de árvore de decisão. Sugestão: chame esse modelo de `clf1` e suas predições de `y_pred1`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import classification_report

X = df.drop(columns='y')
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)

clf1 = DecisionTreeClassifier()

clf1.fit(X_train,y_train)

y_pred1 = clf1.predict(X_test)

accuracy = clf1.score(X_test, y_test)
print('\nScore de Acuracidade (2):\n')
print(f'{accuracy:.2f}')

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred1))


Score de Acuracidade (2):

0.87

Classification Report:

              precision    recall  f1-score   support

           0       0.93      0.92      0.93     11977
           1       0.45      0.47      0.46      1587

    accuracy                           0.87     13564
   macro avg       0.69      0.70      0.69     13564
weighted avg       0.87      0.87      0.87     13564



Q2. Qual a acuracidade total obtida?

Q3. Esse é um problema de classes desbalanceadas? Você pode observar isso pelo classification report?

# Construa um modelo de Árvore de Decisão dos dados normalizados

Aplique a normalização `StandardScaler`,
empregue a separação de dados de treinamento e teste como abaixo e os parâmetros padrão de um modelo de árvore de decisão. Os valores de `y` não serão normalizados. Sugestão: chame esse modelo de `clf2` e suas predições de `y_pred2`.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df.drop(columns='y'))

df_scaled = scaler.transform(df.drop(columns='y'))
df_scaled = pd.DataFrame(df_scaled, columns=df.drop(columns='y').columns)

df_scaled.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,1.606965,-0.103820,-0.275762,1.036362,-0.13549,0.256419,0.893915,-0.436803,1.514306,-1.298476,0.823773,0.011016,-0.569351,-0.411453,-0.25194,0.444898
1,0.288529,1.424008,1.368372,-0.300556,-0.13549,-0.437895,0.893915,-0.436803,1.514306,-1.298476,0.823773,-0.416127,-0.569351,-0.411453,-0.25194,0.444898
2,-0.747384,-0.714951,-0.275762,-0.300556,-0.13549,-0.446762,0.893915,2.289359,1.514306,-1.298476,0.823773,-0.707361,-0.569351,-0.411453,-0.25194,0.444898
3,0.571051,-1.020516,-0.275762,2.373280,-0.13549,0.047205,0.893915,-0.436803,1.514306,-1.298476,0.823773,-0.645231,-0.569351,-0.411453,-0.25194,0.444898
4,-0.747384,2.035139,1.368372,2.373280,-0.13549,-0.447091,-1.118674,-0.436803,1.514306,-1.298476,0.823773,-0.233620,-0.569351,-0.411453,-0.25194,0.444898


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import classification_report

X = df_scaled
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)

clf2 = DecisionTreeClassifier()

clf2.fit(X_train,y_train)

y_pred2 = clf2.predict(X_test)

accuracy = clf2.score(X_test, y_test)
print('\nScore de Acuracidade (2):\n')
print(f'{accuracy:.2f}')

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred2))


Score de Acuracidade (2):

0.87

Classification Report:

              precision    recall  f1-score   support

           0       0.93      0.92      0.93     11977
           1       0.45      0.48      0.46      1587

    accuracy                           0.87     13564
   macro avg       0.69      0.70      0.70     13564
weighted avg       0.87      0.87      0.87     13564



Q4. Qual a acuracidade do novo modelo obtido com os dados normalizados?

Q5. Qual a diferença entre as predições dos modelos não normalizado e normalizado? O que você pode dizer que confirma com esse resultado?

In [ ]:
( len(X_test) - sum(y_pred1 == y_pred2) )/len(X_test)

0.030669419050427603

# Construa um modelo de Árvore de Decisão (dados não normalizados) selecionando apenas os melhores recursos.

Repita o modelo `clf1` empregando agora somente os 8 atributos mais significativos (empregue o critério da ANOVA). Sugestão: chame esse modelo de `clf3` e suas predições de `y_pred3`.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif
X = df.drop(columns='y')
y = df['y']

select_features = SelectKBest(k=8).fit(X, y)
print( X.columns[select_features.get_support()] )

X_selection = select_features.transform(X)

df_selection = pd.DataFrame( X_selection, columns= X.columns[select_features.get_support()] )
df_selection.head()

Index(['housing', 'loan', 'contact', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome'],
      dtype='object')


,housing,loan,contact,duration,campaign,pdays,previous,poutcome
0,1,0,2,261,1,-1,0,3
1,1,0,2,151,1,-1,0,3
2,1,1,2,76,1,-1,0,3
3,1,0,2,92,1,-1,0,3
4,0,0,2,198,1,-1,0,3


In [ ]:
set(df.columns) - set(df_selection.columns)

{'age',
 'balance',
 'day',
 'default',
 'education',
 'job',
 'marital',
 'month',
 'y'}

Q6. Assinale as alternativas em que só aparecem atributos que foram selecionados.



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import classification_report

X = df_selection
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)

clf3 = DecisionTreeClassifier()

clf3.fit(X_train,y_train)

y_pred3 = clf3.predict(X_test)

accuracy = clf3.score(X_test, y_test)
print('\nScore de Acuracidade (2):\n')
print(f'{accuracy:.2f}')

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred3))


Score de Acuracidade (2):

0.87

Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.94      0.93     11977
           1       0.44      0.38      0.41      1587

    accuracy                           0.87     13564
   macro avg       0.68      0.66      0.67     13564
weighted avg       0.86      0.87      0.87     13564



In [ ]:
#@markdown generate bank-new
import pandas as pd

df_new = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/bank-full.csv')
df_new = df_new.drop(columns='y')
df_new = df_new.iloc[[168,44781,34863,5358,35390,24158, 32728, 13395, 39697, 43203]]
df_new = df_new.reset_index(drop=True)
df_new.to_csv('bank-new.csv',index=None)

df_new.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown
1,62,retired,married,tertiary,no,2557,yes,no,cellular,14,sep,211,1,216,7,success
2,31,housemaid,single,tertiary,no,399,yes,no,cellular,6,may,333,1,-1,0,unknown
3,41,management,single,tertiary,no,16,no,no,unknown,23,may,95,1,-1,0,unknown
4,54,blue-collar,married,primary,no,6242,yes,no,cellular,7,may,162,1,-1,0,unknown


Q7. Qual a acuracidade do novo modelo obtido somente com os atributos selecionados?

# Predição

Faça a predição dos novos casos com base no último modelo. Dica: não esqueça de aplicar as mesmas tranformações de encode aplicadas antes.

In [ ]:
df = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/bank-new.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,54,0,1,2,0,184,0,0,2,5,3,673,2,-1,0,1
1,62,4,1,2,0,2557,1,0,0,14,5,211,1,216,7,0
2,31,2,2,2,0,399,1,0,0,6,3,333,1,-1,0,1
3,41,3,2,2,0,16,0,0,2,23,3,95,1,-1,0,1
4,54,1,1,0,0,6242,1,0,0,7,3,162,1,-1,0,1


In [ ]:
for column in df_new.select_dtypes(exclude='number'):
  label_encode = LabelEncoder_dict[column]
  df_new[column] = label_encode.transform(df_new[column])

df_new.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,54,0,1,2,0,184,0,0,2,5,3,673,2,-1,0,1
1,62,4,1,2,0,2557,1,0,0,14,5,211,1,216,7,0
2,31,2,2,2,0,399,1,0,0,6,3,333,1,-1,0,1
3,41,3,2,2,0,16,0,0,2,23,3,95,1,-1,0,1
4,54,1,1,0,0,6242,1,0,0,7,3,162,1,-1,0,1


In [ ]:
clf3.predict(df_new[clf3.feature_names_in_])

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

Q8. Quais as predições obtidas?